# Do not forget to install all dependencies first:
!pip install -Uqq WhisperSpeech==0.8.9 webdataset

https://colab.research.google.com/drive/1xxGlTbwBmaY6GKA24strRixTXGBOlyiw#scrollTo=fdac8407et

In [1]:
import torch
from torch import optim
import torch.nn.functional as F
from datasets import load_dataset
import IPython
import numpy as np
import tensorflow as tf
import datetime
import librosa
import itertools
from torch.autograd import gradcheck

c:\Users\mike\Downloads\deepfake2\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [3]:
# check "7. Pipeline.ipynb"
from whisperspeech import pipeline

In [4]:
pipe = pipeline.Pipeline(t2s_ref='t2s-fast-small-en+pl.model', s2a_ref='s2a-q4-hq-fast-en+pl.model')

c:\Users\mike\Downloads\deepfake2\venv\lib\site-packages\whisperspeech\t2s_up_wds_mlang_enclm.py:365: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  spec = torch.load(local_f

In [5]:
# pipe.generate_to_notebook("""
# I am Whisper Speech, trained on the Juwels supercomputer.
# """)

In [38]:
def getLoss(y: torch.Tensor, y_hat: torch.Tensor):
    # maxShape = min(y.shape[2], y_hat.shape[2])
    # y_hat = y_hat.reshape((y_hat.shape[1]))
    
    y_size = y.shape[2]
    y_hat_size = y_hat.shape[2]

    # minShape = min(y_size, y_hat_size)

    # y = y[:minShape]
    # y_hat = y_hat[:minShape]

    # print(y)
    print(y.shape, y_hat.shape)
    y = F.pad(input=y, pad=(0, max(y_hat_size - y_size, 0), 0, 0, 0, 0), mode='constant', value=0)
    y_hat = F.pad(input=y_hat, pad=(0, max(y_size - y_hat_size, 0), 0, 0, 0, 0), mode='constant', value=0)
    print(y.shape, y_hat.shape)
    
    loss = torch.nn.functional.mse_loss(y, y_hat)

    return loss

In [7]:
dataset = iter(load_dataset("mozilla-foundation/common_voice_17_0", "en", split="train", streaming=True))

In [8]:
sample = next(dataset)

Reading metadata...: 1101170it [00:19, 55788.90it/s]


In [9]:
sentence = sample['sentence']
audio = torch.Tensor(sample['audio']['array'])
sample_rate = sample['audio']['sampling_rate']

In [10]:
# pipe.generate_to_notebook(sentence)

In [11]:
def preprocess(audio, threshold=0.1):
    index = np.argmax(audio > threshold)
    audio = audio[index:][::-1]
    index = np.argmax(audio > threshold)
    audio = audio[index:][::-1]
    audio = librosa.resample(audio, orig_sr=sample_rate, target_sr=24000)

    return audio

In [12]:
audio = torch.Tensor(preprocess(audio.numpy())).cuda()
audio.requires_grad = True
print(audio.shape)

torch.Size([93944])


In [13]:
IPython.display.Audio(audio.cpu().detach().numpy(), rate=24000)

In [14]:
numEpochs = 10

In [40]:
# optimizerT2S = optim.Adam(pipe.t2s.parameters(), lr=0.00001)
# optimizerS2A = optim.Adam(itertools.islice(pipe.s2a.parameters(), 1), lr=0)

pipe.default_speaker.requires_grad = True
# print(pipe.default_speaker)
optimizerSpeaker = optim.LBFGS([torch.nn.Parameter(pipe.default_speaker)], lr=1000)

with torch.no_grad():
    assert pipe.default_speaker.requires_grad
    for name, parameter in pipe.t2s.named_parameters():
        # print(name)
        assert parameter.requires_grad
    for name, parameter in pipe.s2a.named_parameters():
        # print(name)
        assert parameter.requires_grad

In [ ]:
def vocosLoss(y: torch.Tensor, y_hat: torch.Tensor):
    y_hat = y_hat.reshape((y_hat.shape[1]))
    y = y[0]
    
    # print(y.shape, y_hat.shape)

    y_size = y.shape[0]
    y_hat_size = y_hat.shape[0]

    minShape = min(y_size, y_hat_size)

    y = y[:minShape]
    y_hat = y_hat[:minShape]

    # y = F.pad(input=y, pad=(0, max(y_hat_size - y_size, 0)), mode='constant', value=0)
    # y_hat = F.pad(input=y_hat, pad=(0, max(y_size - y_hat_size, 0)), mode='constant', value=0)

    loss = torch.nn.functional.mse_loss(y, y_hat)

    return loss

def evolutionLoss(notes, audio):
    return vocosLoss(pipe.vocoder.decode(notes).detach().to('cpu'), audio.detach().to('cpu'))

def generatePopulation(startingSize, maxNote):
    population = np.random.randint(0, maxNote, size=startingSize)
    return population

population = generatePopulation((10,4,284), 1000)

def crossover(parent1, parent2, maxSize=1000):
    n, m = len(parent1), len(parent1[0])

    child = []

    for i in range(n):
        sub = []
        for j in range(m):
            prob = np.random.random()
            if prob < 0.495:
                sub.append(parent1[i][j])
            elif prob < 0.99:
                sub.append(parent2[i][j])
            else:
                sub.append(int(np.random.randint(0, maxSize)))

        child.append(sub)

    # for i in range(min(n, m), n):
    #     child.append(parent1[i])

    # for i in range(min(n, m), m):
    #     child.append(parent2[i])

    return child

def step(population):
    fitness = []
    n = len(population)

    for x in population:
        # print(x)
        notes = torch.Tensor(np.array(x)).to(dtype=torch.long)
        # print(notes.shape)
        notes = notes.unsqueeze(0)
        # print(notes.shape)
        # print(notes)
        fitness.append(-evolutionLoss(notes, audio))

    fitness = np.array(fitness)
    
    # print(sum(fitness) / n)
    # print(sum(fitness), fitness)
    s = sum(fitness)
    fitness /= s

    newPopulation = []
    # print(fitness)
    for i in range(n):
        p1, p2 = np.random.choice(n, 2, p=fitness)
        
        new = crossover(population[p1], population[p2])
        notes = torch.Tensor(np.array(new)).to(dtype=torch.long)
        # print(notes.shape)
        notes = notes.unsqueeze(0)
        # print(notes.shape)
        # print(notes)
        fitnessNew = -evolutionLoss(notes, audio) / s
        if fitnessNew < fitness[i]:
            newPopulation.append(new)
        else:
            newPopulation.append(population[i])

    getMinLoss(newPopulation)

    return newPopulation

def getMinLoss(population):
    minLoss = (float('inf'), None)
    for x in population:
        notes = torch.Tensor(np.array(x)).to(dtype=torch.long)
        # print(notes.shape)
        notes = notes.unsqueeze(0)
        # print(notes.shape)
        # print(notes)
        loss = evolutionLoss(notes, audio)
        if loss < minLoss[0]:
            minLoss = (loss, notes)

    print(minLoss[0])
    return minLoss[1]

for epoch in range(1):
    if epoch % 100 == 0: print()
    population = step(population)

notes = getMinLoss(population)
audio_hat = pipe.vocoder.decode(notes).detach().to('cpu')



tensor(0.0610)
tensor(0.0610)


In [17]:
import copy
target_atoks = copy.deepcopy(notes)
print(target_atoks)

tensor([[[239, 629, 919,  ..., 635, 516, 629],
         [739,  81, 697,  ..., 172, 538, 455],
         [793, 458, 680,  ..., 874,   4, 290],
         [436, 833, 199,  ..., 555, 303, 175]]])


In [18]:
IPython.display.Audio(audio_hat.cpu().numpy(), rate=24000)

In [19]:
IPython.display.Audio(audio.detach().cpu().numpy(), rate=24000)

In [20]:
from whisperspeech import inference
from torch.profiler import record_function
from fastprogress import progress_bar

def generate_one(toks, positions, langs, xenc, xenc_positions, T, top_k):
    # print(toks.to(dtype=torch.long))
    probs = pipe.s2a(None, toks.to(dtype=torch.long), None, langs, noloss=True, xenc=xenc, xenc_positions=xenc_positions, atoks_positions=positions)
    probs = probs[:,:,-1]
    # print(inference.sample(probs, T, top_k).shape)
    # print(probs)
    # print(inference.sample(probs, T, top_k))
    # print(probs.shape)
    indices = inference.sample(probs, T, top_k)
    # print(indices.shape, probs.shape)
    o = torch.Tensor([probs[0][0][indices[0][0]], probs[0][1][indices[0][1]], probs[0][2][indices[0][2]], probs[0][3][indices[0][3]]]).reshape(indices.shape).to("cuda")
    # print(o.shape, indices.shape)
    # print(indices)
    
    # print(o, o.shape)
    # return o
    return indices

def generate_next(*args, **kwargs):
    return generate_one(*args, **kwargs)

# @torch.no_grad()
def generate(stoks, speakers, langs=None, atoks_prompt=None, N=None, bs=1, T=0.7, top_k=None, show_progress_bar=True, step=None, subsample_enc=False):
    dev = pipe.s2a.device
    N = N or len(stoks) * 3
    stoks = F.pad(stoks.to(dev), (1, pipe.s2a.stoks_len - len(stoks) - 1), value=pipe.s2a.stoks_codes-1).unsqueeze(0)
    speakers = speakers.to(device=dev, dtype=pipe.s2a.dtype)
    toks = torch.full((bs,pipe.s2a.quantizers,pipe.s2a.ctx_n), pipe.s2a.codes+1, dtype=torch.float16, device=dev)
    T = torch.tensor(T, device=dev)

    start = 0 # number of valid tokens or the index of first empty spot
    if atoks_prompt is not None:
        start = atoks_prompt.shape[-1]
        for i in range(pipe.s2a.quantizers):
            toks[:,i,1+i:start+i+1] = atoks_prompt[:,i]
    start += 1 # we always start with at least an SOT

    with record_function("encode"):
        stoks, speakers = [x.repeat(bs, 1) for x in (stoks, speakers)]
        xenc, xenc_positions, _ = pipe.s2a.run_encoder(stoks, speakers)
        toks_positions = torch.arange(N, device=dev)
    with record_function("prefill"):
        initial = generate_one(toks[:,:,:start], toks_positions[:start], langs, xenc, xenc_positions, T, top_k)
        toks[:,:start,start:start+1] = initial[:,:start]
        # print(toks[:,:start,start:start+1], initial[:,:start])
        start += 1
        
    # with inference.inference_context():
    it = range(start,min(N,pipe.s2a.ctx_n-1))
    if show_progress_bar: it = progress_bar(it)

    for i in it:
        with record_function("generate_one"):
            toks[:,:i,i:i+1] = generate_next(toks[:,:,i-1:i], toks_positions[i-1:i], langs, xenc, xenc_positions, T, top_k)[:,:i]

        # for profiling, debugging or early exit
        if step is not None: step()
    
    
    # shift tokens
    toks = toks[:,:,1:N]
    for j in range(pipe.s2a.quantizers):
        toks[:, j] = torch.roll(toks[:, j], -j)
    return toks[:,:,:N-4]

In [21]:
def generate_stoks(text):
    text = text.replace("\n", " ")
    stoks = pipe.t2s.generate(text, cps=15, lang='en', step=None)[0]
    return stoks

# TODO make diffy
def generate_atoks(stoks):
    speaker = pipe.default_speaker
    return generate(stoks=stoks, speakers=speaker.unsqueeze(0), step=None)

In [41]:
from pathlib import Path

sentence = sentence.replace("\n", " ")
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)

stoks = generate_stoks(sentence)

for epoch in range(numEpochs):
    # optimizerT2S.zero_grad()
    # optimizerS2A.zero_grad()
    optimizerSpeaker.zero_grad()

    # audio_hat = pipe.generate(sentence)
    # audio_hat = pipe.generate_atoks(sentence, None, lang='en', cps=15, step_callback=None)
    audio_hat = generate_atoks(stoks)
    
    print(audio_hat)
    print(audio_hat.shape)
    # pipe.vocoder.
    audio_hat.requires_grad = True

    print(target_atoks.shape, audio_hat.shape)
    loss = getLoss(target_atoks.to("cuda"), audio_hat.to("cuda"))
    with train_summary_writer.as_default():
        tf.summary.scalar('loss', loss.item(), step=epoch)
    # loss.requires_grad = True
    loss.backward()
    # loss.backward(pipe.default_speaker)

    # NoneType
    # for name, parameter in pipe.encoder.named_parameters():
    #     if parameter.grad or not parameter.requires_grad:
    #         print(name, parameter.requires_grad, parameter.grad)

    # Vocoder Type
    # for name, parameter in pipe.vocoder:
    #     if parameter.grad or not parameter.requires_grad:
    #         print(name, parameter.requires_grad, parameter.grad)

    for name, parameter in pipe.t2s.named_parameters():
        if parameter.grad or not parameter.requires_grad:
            print(name, parameter.requires_grad, parameter.grad)

    for name, parameter in pipe.s2a.named_parameters():
        if parameter.grad or not parameter.requires_grad:
            print(name, parameter.requires_grad, parameter.grad)

    print("default_speaker", pipe.default_speaker.requires_grad, pipe.default_speaker.grad)
    # print(loss.grad)
    # print(audio.grad)
    print(audio_hat.grad)
    print()
    
    for name, parameter in pipe.s2a.named_parameters():
        print(parameter.grad)


    tmp = pipe.default_speaker.clone()
    optimizerSpeaker.step()

    assert not torch.Tensor.equal(tmp, pipe.default_speaker)
    # optimizerT2S.step()
    # optimizerS2A.step()

C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.9_3.9.3568.0_x64__qbz5n2kfra8p0\lib\contextlib.py:87: FutureWarning: `torch.backends.cuda.sdp_kernel()` is deprecated. In the future, this context manager will be removed. Please see `torch.nn.attention.sdpa_kernel()` for the new context manager, with updated signature.
  self.gen = func(*args, **kwds)


tensor([[[ 865.,  780.,  408.,  ...,  432.,  432.,  432.],
         [ 424.,  424.,  913.,  ...,  571.,  571.,  571.],
         [ 678.,  678.,  989.,  ...,  804.,  936.,  915.],
         [ 673.,  673.,  215.,  ..., 1019.,  825.,  838.]]], device='cuda:0',
       dtype=torch.float16)
torch.Size([1, 4, 281])
torch.Size([1, 4, 284]) torch.Size([1, 4, 281])
torch.Size([1, 4, 284]) torch.Size([1, 4, 281])
torch.Size([1, 4, 284]) torch.Size([1, 4, 284])


RuntimeError: Found dtype Long but expected Half

In [ ]:
audio = pipe.generate(sentence)

In [ ]:
IPython.display.Audio(audio.cpu().numpy(), rate=24000)

In [ ]:
IPython.display.Audio(audio_hat.cpu().numpy(), rate=24000)